In [234]:
import random
import torch
import time

# Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9) и задавались следующим образом (25/50/75/100):
x - последовательность цифр  
y1 = x1, y(i) = x(i) + x(1).  
Если y(i) >= 10, то y(i) = y(i) - 10

In [235]:
vocab = [int(char) for char in '0123456789']

In [254]:
num_examples = 128
message_length = 5


def dataset(num_examples):
    dataset = []
    
    for ex in range(num_examples):     
        x = [random.choice(vocab) for x in range(message_length)]
        
        y = [x[0]]
        for xi in x[1:]:
            if (xi + x[0]) >= 10:
                y.append((xi + x[0])%10)
            else:
                y.append(xi + x[0])        
        
        dataset.append([torch.tensor(x), torch.tensor(y)])
    return dataset

In [255]:
dataset(num_examples)

[[tensor([3, 1, 5, 8, 0]), tensor([3, 4, 8, 1, 3])],
 [tensor([4, 2, 9, 5, 6]), tensor([4, 6, 3, 9, 0])],
 [tensor([4, 9, 2, 9, 0]), tensor([4, 3, 6, 3, 4])],
 [tensor([0, 7, 4, 0, 5]), tensor([0, 7, 4, 0, 5])],
 [tensor([4, 7, 5, 9, 6]), tensor([4, 1, 9, 3, 0])],
 [tensor([0, 6, 0, 2, 4]), tensor([0, 6, 0, 2, 4])],
 [tensor([3, 4, 1, 6, 0]), tensor([3, 7, 4, 9, 3])],
 [tensor([6, 2, 3, 8, 5]), tensor([6, 8, 9, 4, 1])],
 [tensor([3, 1, 0, 8, 5]), tensor([3, 4, 3, 1, 8])],
 [tensor([1, 9, 6, 5, 9]), tensor([1, 0, 7, 6, 0])],
 [tensor([4, 5, 4, 8, 8]), tensor([4, 9, 8, 2, 2])],
 [tensor([7, 2, 1, 3, 1]), tensor([7, 9, 8, 0, 8])],
 [tensor([9, 2, 9, 1, 0]), tensor([9, 1, 8, 0, 9])],
 [tensor([0, 8, 3, 9, 0]), tensor([0, 8, 3, 9, 0])],
 [tensor([5, 1, 2, 0, 2]), tensor([5, 6, 7, 5, 7])],
 [tensor([4, 2, 6, 2, 2]), tensor([4, 6, 0, 6, 6])],
 [tensor([1, 2, 2, 3, 2]), tensor([1, 3, 3, 4, 3])],
 [tensor([1, 0, 9, 4, 0]), tensor([1, 1, 0, 5, 1])],
 [tensor([6, 2, 1, 6, 4]), tensor([6, 8, 7, 2,

# Задача: научить модель предсказывать y(i) по x(i)

In [238]:
embedding_dim = 10
hidden_dim = 10
vocab_size = len(vocab)

num_epochs = 100

## RNN

In [239]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        ## Здесь создать слои
        self.embed = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.linear = torch.nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [240]:
model = Network()

In [241]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.05)

In [242]:
for ep in range(num_epochs):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for x, y in dataset(num_examples):
        
        optimizer.zero_grad()
        answers = model.forward(x.unsqueeze(1))
        answers = answers.view(-1, vocab_size)
        loss = criterion(answers, y)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 0.238, Train loss: 2.398
Epoch 1. Time: 0.222, Train loss: 2.348
Epoch 2. Time: 0.224, Train loss: 2.352
Epoch 3. Time: 0.229, Train loss: 2.335
Epoch 4. Time: 0.225, Train loss: 2.329
Epoch 5. Time: 0.243, Train loss: 2.340
Epoch 6. Time: 0.227, Train loss: 2.356
Epoch 7. Time: 0.219, Train loss: 2.352
Epoch 8. Time: 0.230, Train loss: 2.337
Epoch 9. Time: 0.233, Train loss: 2.324
Epoch 10. Time: 0.235, Train loss: 2.342
Epoch 11. Time: 0.229, Train loss: 2.321
Epoch 12. Time: 0.222, Train loss: 2.334
Epoch 13. Time: 0.220, Train loss: 2.334
Epoch 14. Time: 0.223, Train loss: 2.333
Epoch 15. Time: 0.218, Train loss: 2.334
Epoch 16. Time: 0.220, Train loss: 2.343
Epoch 17. Time: 0.221, Train loss: 2.338
Epoch 18. Time: 0.217, Train loss: 2.338
Epoch 19. Time: 0.226, Train loss: 2.341
Epoch 20. Time: 0.221, Train loss: 2.331
Epoch 21. Time: 0.219, Train loss: 2.335
Epoch 22. Time: 0.219, Train loss: 2.336
Epoch 23. Time: 0.220, Train loss: 2.337
Epoch 24. Time: 0.220, Tra

In [243]:
with torch.no_grad():
        matches, total = 0, 0
        for x, y in dataset(num_examples):
            answers = model.forward(x.unsqueeze(1))
            predictions = torch.nn.functional.softmax(answers, dim=2)
            _, batch_out = predictions.max(dim=2)
            batch_out = batch_out.squeeze(1)
            matches += torch.eq(batch_out, y).sum().item()
            total += torch.numel(batch_out)
        accuracy = matches / total
        print('Accuracy: {:4.2f}%'.format(accuracy * 100))

Accuracy: 9.76%


## GRU

In [244]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        ## Здесь создать слои
        self.embed = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = torch.nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [245]:
model = Network()

In [246]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.05)

In [247]:
for ep in range(num_epochs):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for x, y in dataset(num_examples):
        
        optimizer.zero_grad()
        answers = model.forward(x.unsqueeze(1))
        answers = answers.view(-1, vocab_size)
        loss = criterion(answers, y)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 0.263, Train loss: 2.322
Epoch 1. Time: 0.260, Train loss: 2.307
Epoch 2. Time: 0.257, Train loss: 2.311
Epoch 3. Time: 0.256, Train loss: 2.309
Epoch 4. Time: 0.258, Train loss: 2.306
Epoch 5. Time: 0.257, Train loss: 2.306
Epoch 6. Time: 0.260, Train loss: 2.307
Epoch 7. Time: 0.258, Train loss: 2.307
Epoch 8. Time: 0.256, Train loss: 2.307
Epoch 9. Time: 0.259, Train loss: 2.306
Epoch 10. Time: 0.256, Train loss: 2.306
Epoch 11. Time: 0.256, Train loss: 2.306
Epoch 12. Time: 0.255, Train loss: 2.306
Epoch 13. Time: 0.259, Train loss: 2.307
Epoch 14. Time: 0.256, Train loss: 2.306
Epoch 15. Time: 0.258, Train loss: 2.306
Epoch 16. Time: 0.257, Train loss: 2.306
Epoch 17. Time: 0.260, Train loss: 2.306
Epoch 18. Time: 0.254, Train loss: 2.308
Epoch 19. Time: 0.257, Train loss: 2.307
Epoch 20. Time: 0.256, Train loss: 2.312
Epoch 21. Time: 0.259, Train loss: 2.343
Epoch 22. Time: 0.255, Train loss: 2.306
Epoch 23. Time: 0.257, Train loss: 2.306
Epoch 24. Time: 0.258, Tra

In [248]:
with torch.no_grad():
        matches, total = 0, 0
        for x, y in dataset(num_examples):
            answers = model.forward(x.unsqueeze(1))
            predictions = torch.nn.functional.softmax(answers, dim=2)
            _, batch_out = predictions.max(dim=2)
            batch_out = batch_out.squeeze(1)
            matches += torch.eq(batch_out, y).sum().item()
            total += torch.numel(batch_out)
        accuracy = matches / total
        print('Accuracy: {:4.2f}%'.format(accuracy * 100))

Accuracy: 9.59%


## LSTM

In [249]:
embed = torch.nn.Embedding(vocab_size, embedding_dim)
lstm = torch.nn.LSTM(embedding_dim, hidden_dim)
linear = torch.nn.Linear(hidden_dim, vocab_size)
softmax = torch.nn.functional.softmax
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(embed.parameters()) +
                             list(lstm.parameters()) +
                             list(linear.parameters()), lr=0.001)

In [250]:
def zero_hidden():
    return (torch.zeros(1, 1, hidden_dim),
            torch.zeros(1, 1, hidden_dim))

In [251]:
accuracies, max_accuracy = [], 0
for ep in range(num_epochs):
    print('Epoch: {}'.format(ep))
    for x, y in dataset(num_examples):
        lstm_in = embed(x)
        lstm_in = lstm_in.unsqueeze(1)
        lstm_out, lstm_hidden = lstm(lstm_in, zero_hidden())
        scores = linear(lstm_out)
        scores = scores.transpose(1, 2)
        y = y.unsqueeze(1)
        loss = loss_fn(scores, y) 
        loss.backward()
        optimizer.step()
    print('Loss: {:6.4f}'.format(loss.item()))

Epoch: 0
Loss: 2.3290
Epoch: 1
Loss: 2.2056
Epoch: 2
Loss: 2.3470
Epoch: 3
Loss: 2.2497
Epoch: 4
Loss: 2.3034
Epoch: 5
Loss: 2.3193
Epoch: 6
Loss: 2.3088
Epoch: 7
Loss: 2.3051
Epoch: 8
Loss: 2.3012
Epoch: 9
Loss: 2.3031
Epoch: 10
Loss: 2.2789
Epoch: 11
Loss: 2.3381
Epoch: 12
Loss: 2.3098
Epoch: 13
Loss: 2.3160
Epoch: 14
Loss: 2.3141
Epoch: 15
Loss: 2.3315
Epoch: 16
Loss: 2.3067
Epoch: 17
Loss: 2.2952
Epoch: 18
Loss: 2.3108
Epoch: 19
Loss: 2.3077
Epoch: 20
Loss: 2.3212
Epoch: 21
Loss: 2.2839
Epoch: 22
Loss: 2.3087
Epoch: 23
Loss: 2.3056
Epoch: 24
Loss: 2.3212
Epoch: 25
Loss: 2.3234
Epoch: 26
Loss: 2.3015
Epoch: 27
Loss: 2.3128
Epoch: 28
Loss: 2.3027
Epoch: 29
Loss: 2.2990
Epoch: 30
Loss: 2.3372
Epoch: 31
Loss: 2.2829
Epoch: 32
Loss: 2.3022
Epoch: 33
Loss: 2.3302
Epoch: 34
Loss: 2.3651
Epoch: 35
Loss: 2.3033
Epoch: 36
Loss: 2.3054
Epoch: 37
Loss: 2.3106
Epoch: 38
Loss: 2.3374
Epoch: 39
Loss: 2.3097
Epoch: 40
Loss: 2.3050
Epoch: 41
Loss: 2.3017
Epoch: 42
Loss: 2.3677
Epoch: 43
Loss: 2.291

In [252]:
with torch.no_grad():
        matches, total = 0, 0
        for x, y in dataset(num_examples):
            lstm_in = embed(x)
            lstm_in = lstm_in.unsqueeze(1)
            lstm_out, lstm_hidden = lstm(lstm_in, zero_hidden())
            scores = linear(lstm_out)
            predictions = softmax(scores, dim=2)
            _, batch_out = predictions.max(dim=2)
            batch_out = batch_out.squeeze(1)
            matches += torch.eq(batch_out, y).sum().item()
            total += torch.numel(batch_out)
        accuracy = matches / total
        print('Accuracy: {:4.2f}%'.format(accuracy * 100))

Accuracy: 9.62%


len 25  100e RNN_10.84% GRU 10.44% LSTM 10.38%  
len 50  100e RNN 10.75% GRU 10.50% LSTM 10.55%  
len 75  100e RNN 9.35% GRU 10.00% LSTM 10.26%  
len 100 100e RNN 9.76% GRU 9.59% LSTM  9.62%